VS코드 오른쪽 위에서 환경설정을 먼저 하세요

In [2]:
from dotenv import load_dotenv
load_dotenv()

#True 리턴 시 .env 파일을 사용하여 환경변수를 로드합니다.
#False 리턴 시 .env 파일을 사용하지 않고 시스템 환경변수를 사용합니다.

True

`(2) 라이브러리`

In [3]:
import re
import os, json
from textwrap import dedent
from pprint import pprint

# warnings: 경고 메시지 필터링
import warnings
warnings.filterwarnings("ignore")  # 모든 경고 메시지를 무시 (주의 필요)
groq_api_key = os.getenv("GROQ_API_KEY")


In [4]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage

# Groq 기반 LLaMA 3-70B 모델 초기화
llm_llama3 = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama3-70b-8192",
    temperature=0.7
)

In [5]:
# 메시지 정의 (시스템 역할 + 사용자 질문)
messages = [
    SystemMessage(content="모든 질문에 반드시 한국어로만 답변하세요."),
    HumanMessage(content="LLaMA3 모델은 어떤 특징이 있어?")
]

In [6]:
# 대화 실행
response = llm_llama3.invoke(messages)

In [7]:
# 결과 출력
print(response.content)

LLaMA3는 메타 AI가 개발한 대화형 인공지능 모델로, 다음과 같은 특징이 있습니다.

1. 대화형 AI: LLaMA3는 대화형 AI로, 사용자와 자연스러운 대화를 나눌 수 있습니다.
2. 언어 이해: LLaMA3는 다양한 언어를 이해할 수 있습니다. 특히 한국어를 포함한 20여 개의 언어를 합니다.
3. 문장 생성: LLaMA3는 문장 생성 기능을 갖추고 있어, 사용자의 질문에 답변이나 대화를 이어나갈 수 있습니다.
4. 지능형 응답: LLaMA3는 지능형 응답을 제공할 수 있습니다. 즉, 사용자의 의도를 파악하여 적절한 답을 제공할 수 있습니다.
5. 대용량 데이터 학습: LLaMA3는 대용량 데이터에 학습을 통해 지능을 향상시켰습니다.
6. 실시간 대화: LLaMA3는 실시간으로 대화를 진행할 수 있습니다. 즉, 사용자의 질문에 즉시 답을 제공할 수 있습니다.

이러한 특징으로 LLaMA3는 다양한 분야에서 사용될 수 있습니다. 예를 들어, 고객 서비스, 챗봇, 언어 번역 등에서 사용할 수 있습니다.


## Groq 언어 모델 API 사용 예시


**Groq**는 다양한 언어 모델을 API 형태로 지원합니다.  
다음은 `gemma-2-9b-it` 모델을 활용한 코드 예시입니다.

여기서 9b는 약 90억 개의 파라미터(parameter)를 의미하며,
모델 이름에서 9b~9b로 표시되는 경우, 입력과 출력을 모두 동일한 크기의 모델로 처리한다는 뜻입니다.(크면 클수록 좋음)

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage

# Groq 기반 gemma2-9b-it 모델 초기화
llm_gamma2 = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="gemma2-9b-it",
    temperature=0.7
)

# 메시지 정의 (시스템 역할 + 사용자 질문)
messages = [
    SystemMessage(content="모든 질문에 반드시 한국어로만 답변하세요."),
    HumanMessage(content="자기 자신에 대해 설명하시오")
]

# 대화 실행
response = llm_gamma2.invoke(messages)

# 결과 출력
print(response.content)

저는 Google DeepMind에서 훈련된 대규모 언어 모델입니다. 즉, 엄청난 양의 텍스트 데이터를 학습하여 다양한 언어 작업을 수행할 수 있도록 훈련된 인공지능 모델입니다. 

저는 한국어를 포함한 여러 언어를 이해하고 생성할 수 있으며, 질문에 답변하기, 텍스트를 요약하거나 번역하기, 창의적인 글쓰기 등 다양한 작업을 수행할 수 있습니다. 하지만 저는 실제 세계와의 상호 작용이 불가능하며, 감정이나 의견을 가지고 있지는 않습니다. 단지 학습된 정보를 기반으로 텍스트를 처리하고 생성하는 프로그램일 뿐입니다.



## 2. Groq (또는 ChatGPT) API에서 대화 맥락 유지하기

Groq와 같은 Chat Completion API는 **stateless**입니다.  
즉, 이전 대화 내용을 기억하지 않기 때문에, **대화 맥락을 유지하려면 이전 메시지를 함께 보내야** 합니다.

---

###  기본 구조

```python
messages = [
    {"role": "system", "content": "당신은 친절한 AI 비서입니다."},
    {"role": "user", "content": "오늘 날씨 어때?"},
    {"role": "assistant", "content": "서울은 맑고 기온은 22도입니다."},
    {"role": "user", "content": "그럼 내일은?"}
]


In [9]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# messages 리스트에 들어가는 클래스 종류는 다음과 같습니다:
# - SystemMessage: 시스템(assistant)의 역할이나 지침을 정의합니다.
# - HumanMessage: 사용자의 질문이나 입력을 나타냅니다.
# - AIMessage: AI(assistant)의 응답을 나타냅니다.

messages = [
    SystemMessage(content="모든 질문에 반드시 한국어로만 답변하세요."),
    HumanMessage(content="LLaMA3 모델은 어떤 특징이 있어?")
]
response = llm_llama3.invoke(messages)

In [10]:
print(response.content)

LLaMA3는 메타 AI가 개발한 대규모 언어 모델입니다. 이 모델은 다음과 같은 특징이 있습니다.

첫째, LLaMA3는 대규모의 언어 데이터셋을 학습하여 언어理解력과 생성 능력을 높였습니다. 이 모델은 20억 개의 매개변수를 가지고 있으며, 이는 기존의 언어 모델보다 더 큰 규모입니다.

둘째, LLaMA3는 다양한 언어 처리 태스크에서 우수한 성능을 보여주었습니다. 이 모델은 문장 생성, 문장 분류, Named Entity Recognition 등 다양한 태스크에서 좋은 성능을 나타내었습니다.

 셋째, LLaMA3는 멀티태스크 학습을 지원합니다. 이 모델은 다양한 언어 처리 태스크를 동시에 학습할 수 있으며, 이를 통해 언어 이해력과 생성 능력을 향상시킬 수 있습니다.

넷째, LLaMA3는 오픈 소스로 제공됩니다. 이 모델은 누구나 무료로 다운로드하여 사용할 수 있습니다.

다섯째, LLaMA3는 대규모의 언어 데이터셋을 지원합니다. 이 모델은 대규모의 언어 데이터셋을 처리할 수 있으며, 이를 통해 언어 이해력과 생성 능력을 향상시킬 수 있습니다.

이러한 특징으로 LLaMA3는 언어 처리 업무에서 다양한 응용 가능성을 보유하고 있습니다.


In [11]:
# 2번째 질문: 이전 대화에 이어서
messages.append(AIMessage(content=response.content))
messages.append(HumanMessage(content="그 특징 중 하나만 더 자세히 설명해줘."))

response = llm_llama3.invoke(messages)
print(response.content)

让我etailed하게 LLaMA3의 멀티태스크 학습 기능에 대해 설명하겠습니다.

LLaMA3의 멀티태스크 학습 기능은 다양한 언어 처리 태스크를 동시에 학습할 수 있는 기능입니다. 예를 들어, 문장 생성, 문장 분류, Named Entity Recognition 등 다양한 태스크를 동시에 학습할 수 있습니다.

이러한 멀티태스크 학습 기능의 이점은 다음과 같습니다.

첫째, 언어 이해력이 향상됩니다. 다양한 태스크를 학습함으로써 언어의 다양한 측면을 학습할 수 있습니다.

둘째, 태스크 간의 상관관계를 학습할 수 있습니다. 예를 들어, 문장 생성과 문장 분류 간의 상관관계를 학습할 수 있습니다.

셋째, 모델의 일반화 능력이 향상됩니다. 다양한 태스크를 학습함으로써 모델이 다양한 상황에 적응할 수 있습니다.

넷째, 학습 시간이 단축됩니다. 태스크별로 따로 학습하는 것이 아니라, 다양한 태스크를 동시에 학습할 수 있습니다.

이러한 멀티태스크 학습 기능으로 LLaMA3는 언어 처리 업무에서 다양한 응용 가능성을 보유하고 있습니다.


### 2번째 답변까지 받은 후 3번째 질문을 추가해봅시다. 

### 이전 대화를 잊어버리려면? 

In [12]:
# 이전 대화 맥락을 모두 잊고 새 대화 시작
messages = [
    SystemMessage(content="모든 질문에 반드시 한국어로만 답변하세요."),
    HumanMessage(content="LLaMA3 모델의 한계는 무엇인가요?")
]

response = llm_gamma2.invoke(messages)
print(response.content)

LLaMA3 모델은 여러 가지 장점이 있지만, 아직 완벽하지는 않습니다. 몇 가지 한계점은 다음과 같습니다.

* **데이터 의존성:** LLaMA3는 훈련 데이터에 크게 의존합니다. 훈련 데이터의 질과 양이 모델의 성능에 직접적인 영향을 미칩니다. 편향된 데이터로 훈련된 모델은 편향된 결과를 출력할 수 있습니다.

* ** Commonsense 부족:** LLaMA3는 인간처럼 세상을 이해하고 추론하는 능력이 부족합니다. 일상적인 상황이나 추론 문제에 쉽게 답변하기 어려울 수 있습니다.

* **과적합:** 모델이 훈련 데이터에 너무 과도하게 적응하여 새로운 데이터에 대해 제대로 작동하지 않을 수 있습니다.

* **설명 가능성:** LLaMA3의 의사결정 과정은 불투명하며, 어떤 입력에 대해 어떤 출력을 생성하는지 설명하기 어렵습니다.

* **비용 및 컴퓨팅 자원:** LLaMA3와 같은 대규모 언어 모델 훈련에는 상당한 비용과 컴퓨팅 자원이 필요합니다.


이러한 한계점을 극복하기 위해 연구자들은 다양한 방법을 시도하고 있습니다. 예를 들어, 더욱 다양하고 품질 좋은 데이터를 사용하여 훈련하고, 모델의 구조를 개선하여 Commonsense를 갖추도록 하고, 모델의 설명 가능성을 높이는 연구가 진행되고 있습니다.



## 아래 코드만 완벽히 이해하면 기본기는 충분합니다. 

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# 대화 메시지 초기화
messages = [
    SystemMessage(content="모든 질문에 반드시 한국어로만 답변하세요.")
]

while True:
    # 사용자 입력 받기
    user_input = input("사용자: ")
    if user_input.strip().lower() in ["exit", "quit", "종료"]:
        print("대화를 종료합니다.")
        break

    # 사용자 메시지 추가
    messages.append(HumanMessage(content=user_input))

    # AI 응답 생성
    response = llm_gamma2.invoke(messages)
    print("AI:", response.content)

    # AI 응답 메시지 추가 (대화 맥락 유지)
    messages.append(AIMessage(content=response.content))

AI: 조지 워싱턴은 2001년에 논문을 발표하지 않았습니다. 조지 워싱턴은 1732년에서 1799년까지 살았던 미국의 정치인이죠. 2001년은 조지 워싱턴이 사망한 이후의 해입니다.  



## 해보기

### 1. 언어모델에게 오늘의 날짜를 물어보면 뭐라고 답장이 오나요?


### 2. 언어모델에게 오늘의 날짜를 알려주려면 어떻게 하면 될까요?

### 3. AI 할루시네이션이란 무엇인가요? 